NoSQL datapipeline
Apache Airflow - Orchestrador e Agendador
OpenWeatherAPI - api de tempo
Python (extrair api)
Pyspark - Transformar, validar e carregar
MongoDB
Criaçao de Dashboard interativo

In [6]:
%cd /content/drive/MyDrive/github_projects/openweather-pyspark-nosql

/content/drive/MyDrive/github_projects/openweather-pyspark-nosql


In [7]:
!pip install python-dotenv

In [13]:
import json
import pandas as pd

In [9]:
from mnt.python.coletor_weather import ColetorWeather

# Lista de cidades (ex: capitais)
cidades = [
    'Rio Branco,BR',        # Acre (AC)
    'Maceió,BR',               # Alagoas (AL)
    'Macapá,BR',               # Amapá (AP)
    'Manaus,BR',               # Amazonas (AM)
    'Salvador,BR',             # Bahia (BA)
    'Fortaleza,BR',            # Ceará (CE)
    'Brasília,BR',             # Distrito Federal (DF)
    'Vitória,BR',              # Espírito Santo (ES)
    'Goiânia,BR',              # Goiás (GO)
    'São Luís,BR',             # Maranhão (MA)
    'Cuiabá,BR',               # Mato Grosso (MT)
    'Campo Grande,BR',         # Mato Grosso do Sul (MS)
    'Belo Horizonte,BR',       # Minas Gerais (MG)
    'Belém,BR',                # Pará (PA)
    'João Pessoa,BR',          # Paraíba (PB)
    'Curitiba,BR',             # Paraná (PR)
    'Recife,BR',               # Pernambuco (PE)
    'Teresina,BR',             # Piauí (PI)
    'Rio de Janeiro,BR',       # Rio de Janeiro (RJ)
    'Natal,BR',                # Rio Grande do Norte (RN)
    'Porto Alegre,BR',         # Rio Grande do Sul (RS)
    'Porto Velho,BR',          # Rondônia (RO)
    'Boa Vista,BR',            # Roraima (RR)
    'Florianópolis,BR',        # Santa Catarina (SC)
    'São Paulo,BR',            # São Paulo (SP)
    'Aracaju,BR',              # Sergipe (SE)
    'Palmas,BR'                # Tocantins (TO)
]

# Instancia e coleta
extract = ColetorWeather(cidades)
dados = extract.coletar()

# Salvar dados
extract.salvar_jsons(dados)



[✓] Coletado: Rio Branco,BR
[✓] Coletado: Maceió,BR
[✓] Coletado: Macapá,BR
[✓] Coletado: Manaus,BR
[✓] Coletado: Salvador,BR
[✓] Coletado: Fortaleza,BR
[✓] Coletado: Brasília,BR
[✓] Coletado: Vitória,BR
[✓] Coletado: Goiânia,BR
[✓] Coletado: São Luís,BR
[✓] Coletado: Cuiabá,BR
[✓] Coletado: Campo Grande,BR
[✓] Coletado: Belo Horizonte,BR
[✓] Coletado: Belém,BR
[✓] Coletado: João Pessoa,BR
[✓] Coletado: Curitiba,BR
[✓] Coletado: Recife,BR
[✓] Coletado: Teresina,BR
[✓] Coletado: Rio de Janeiro,BR
[✓] Coletado: Natal,BR
[✓] Coletado: Porto Alegre,BR
[✓] Coletado: Porto Velho,BR
[✓] Coletado: Boa Vista,BR
[✓] Coletado: Florianópolis,BR
[✓] Coletado: São Paulo,BR
[✓] Coletado: Aracaju,BR
[✓] Coletado: Palmas,BR
[✓] Total de cidades coletadas: 27
[✓] Arquivos salvos em: /content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data/raw/ano=2025/mes=05/dia=24/hora=14


In [15]:
with open('/content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data/raw/ano=2025/mes=05/dia=24/hora=14/clima_Aracaju_2025-05-24_14-04-22.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

df = pd.json_normalize(dados)

display(df)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.sea_level,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1748095462,-10800,3471872,Aracaju,200,-37.0717,-10.9111,...,1017,1015,6.17,130,75,1,8322,BR,1748075910,1748117527


In [33]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (74.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 126102 files and directories currently instal

In [16]:
!tree -L 2

.
├── data
│   └── raw
├── docker
│   ├── airflow
│   ├── notebook
│   └── requirements.txt
├── info.txt
├── mnt
│   ├── airflow
│   ├── notebook
│   └── python
├── README.md
└── services

10 directories, 3 files


In [36]:
!pip install pyspark findspark

In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, when, current_timestamp, sha2, concat_ws

In [39]:
# Inicializa Spark
spark = SparkSession.builder \
    .appName("Transformacao_Clima_OpenWeather") \
    .getOrCreate()

In [43]:
# Caminho da pasta onde estão os arquivos JSON
path = "/content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data"

# Lê todos os arquivos JSON da pasta
df_raw = spark.read.option("multiLine", True).json(f"{path}/*.json")

df_raw.printSchema()

root
 |-- base: string (nullable = true)
 |-- clouds: struct (nullable = true)
 |    |-- all: long (nullable = true)
 |-- cod: long (nullable = true)
 |-- coord: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lon: double (nullable = true)
 |-- dt: long (nullable = true)
 |-- id: long (nullable = true)
 |-- main: struct (nullable = true)
 |    |-- feels_like: double (nullable = true)
 |    |-- grnd_level: long (nullable = true)
 |    |-- humidity: long (nullable = true)
 |    |-- pressure: long (nullable = true)
 |    |-- sea_level: long (nullable = true)
 |    |-- temp: double (nullable = true)
 |    |-- temp_max: double (nullable = true)
 |    |-- temp_min: double (nullable = true)
 |-- name: string (nullable = true)
 |-- rain: struct (nullable = true)
 |    |-- 1h: double (nullable = true)
 |-- sys: struct (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- sunrise: long (nullable = true)
 |    |-- s

In [48]:
# Transformações + enriquecimento
df_transformado = df_raw.select(
    col("name").alias("cidade"),
    col("dt").alias("timestamp"),
    col("main.temp").alias("temperatura"),
    col("main.humidity").alias("umidade"),
    col("wind.speed").alias("velocidade_vento"),
    col("weather")[0]["description"].alias("descricao_clima")
).withColumn(
    "data_hora", from_unixtime(col("timestamp"))
).withColumn(
    "faixa_temperatura",
    when(col("temperatura") < 15, "Frio")
    .when(col("temperatura") < 30, "Agradável")
    .otherwise("Quente")
).withColumn(
    "data_coleta", current_timestamp()
).filter(
    (col("temperatura").isNotNull()) &
    (col("temperatura") > -30) & (col("temperatura") < 50)
).na.fill({
    "umidade": 0,
    "velocidade_vento": 0,
    "descricao_clima": "não informado"
}).dropDuplicates(["cidade", "timestamp"])

# Gerar _id único para evitar duplicatas no MongoDB
df_validado = df_transformado.withColumn(
    "_id",
    sha2(concat_ws("_", col("cidade"), col("timestamp")), 256)
)

# Verifica os dados finais
df_validado.printSchema()
df_validado.show(100)

root
 |-- cidade: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- temperatura: double (nullable = true)
 |-- umidade: long (nullable = false)
 |-- velocidade_vento: double (nullable = false)
 |-- descricao_clima: string (nullable = false)
 |-- data_hora: string (nullable = true)
 |-- faixa_temperatura: string (nullable = false)
 |-- data_coleta: timestamp (nullable = false)
 |-- _id: string (nullable = true)

+--------------+----------+-----------+-------+----------------+-----------------+-------------------+-----------------+--------------------+--------------------+
|        cidade| timestamp|temperatura|umidade|velocidade_vento|  descricao_clima|          data_hora|faixa_temperatura|         data_coleta|                 _id|
+--------------+----------+-----------+-------+----------------+-----------------+-------------------+-----------------+--------------------+--------------------+
|       Aracaju|1748090793|      26.97|     83|            5.14| nuvens quebr

In [53]:
!rm -rf "/content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data"